In [50]:
import os
import json
import geojson
import pandas as pd
import pymongo
from pprint import pprint
import numpy as np
from flask import Flask, render_template, jsonify

In [32]:
# path constructor
def path_constructor(parent, child):
    return os.path.join(parent, child)

In [33]:
# data folder
dataFolder = path_constructor("static", "data")
jsonFolder=path_constructor(dataFolder,"json")
csvFolder=path_constructor(dataFolder,"csv")
jsonFiles = os.listdir(jsonFolder)

In [34]:
# json files path
jsonFilespath = []
for j in jsonFiles:
    path = path_constructor(jsonFolder, j)
    jsonFilespath.append(path)

In [35]:
# Audio files used in the ML project
bird_df = pd.read_csv(path_constructor(csvFolder, "bird_calls.csv"))
bird_df.head()

,fname,label
0,American_Goldfinch_XC114342.wav,American_Goldfinch
1,American_Goldfinch_XC124312.wav,American_Goldfinch
2,American_Goldfinch_XC133564.wav,American_Goldfinch
3,American_Goldfinch_XC133565.wav,American_Goldfinch
4,American_Goldfinch_XC141469.wav,American_Goldfinch


In [36]:
# file IDs
birdID = bird_df.fname.str[-10:-4]
birdList=list(np.unique(bird_df.label))

In [23]:
[print(a, b) for a, b in zip(jsonFilespath,birdList)]

static\data\json\American_Goldfinch.json American_Goldfinch
static\data\json\American_Robin.json American_Robin
static\data\json\Barn_Swallow.json Barn_Swallow
static\data\json\Blue-grey_Gnatcatcher.json Blue-grey_Gnatcatcher
static\data\json\Blue_Jay.json Blue_Jay
static\data\json\Broad-winged_Hawk.json Carolina_Chickadee
static\data\json\Carolina_Chickadee.json Carolina_Wren
static\data\json\Carolina_Wren.json Cedar_Waxwing
static\data\json\Cedar_Waxwing.json Northern_Cardinal
static\data\json\Eastern_Bluebird.json Ruby-crowned_Kinglet


[None, None, None, None, None, None, None, None, None, None]

In [51]:
# Set up mongo connection
conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)

In [52]:
# Connect to mongo db and collection
db = client.Birds
collection = db.BirdCall

In [53]:
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Birds'), 'BirdCall')

In [54]:
# Function to insert the json data in a collection
def insertData(pathList, collection):    
    for path in pathList:
        with open(path, 'r', encoding='utf-8') as f:
            contents = json.loads(f.read())
            collection.insert_many(contents)

In [55]:
# Insert raw data in the collection
for path,b in zip(jsonFilespath, birdList):
    with open(path, 'r', encoding='utf-8') as f:
        contents= json.load(f)
        data={b:contents}
        collection.insert_many(contents)
        

TypeError: document must be an instance of dict, bson.son.SON, bson.raw_bson.RawBSONDocument, or a type that inherits from collections.MutableMapping

In [21]:
def queryBirds(birdName, birdCollection):
    birdData = [b for b in birdCollection.find({"recordings":{"en": birdName}})]          
    return birdData